# Indices

In [3]:
import pandas as pd
import unicodedata
from main import load_json

def remove_accents(input_str: str) -> str:
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])



In [4]:
def slugfy(texto: str) -> str:
    return remove_accents(texto).replace(" ", "-").replace(".", "").lower()

In [5]:
df = pd.read_csv("dados/saude-series.csv") 
df.referencia = df.referencia.apply( lambda x: pd.to_datetime(x, format="%Y%m"))


In [6]:
mapa_ibge = {int(key): value for (key, value) in load_json("dados/ibge.json").items()}

In [7]:
df.Ibge = df.Ibge.apply(lambda x: mapa_ibge[x])

In [8]:
indices = df.copy()

In [9]:
for x in indices.columns[1:-3]:
    indices[x] = indices[x]/indices.população*1000

In [10]:
indices.to_pickle("dados/indices.pd.pkl")

In [11]:
df.to_pickle("dados/saude-series.pd.pkl")

In [12]:
df.query(f"Ibge == 4202909")[["referencia","Asma"]].rename(columns={"Asma": "atendimentos"})

,referencia,atendimentos
2,2013-08-01,0
10,2013-09-01,1
18,2013-10-01,0
35,2014-01-01,0
41,2014-02-01,0
...,...,...
21356,2021-10-01,130
21610,2021-11-01,81
21940,2021-12-01,59
22193,2022-01-01,78


# Cidades

In [ ]:
ibge_muni = [(x["pk"], x["fields"]["nome"]) for x in municipios]

with open("dados/lista-ibge-municipio.json","w") as fp:
    json.dump(ibge_muni, fp, ensure_ascii=False)

In [ ]:
ibge_regi = [(x["pk"], x["fields"]["regiao"]) for x in municipios]


In [ ]:
municipios = pd.DataFrame([(x["pk"], x["fields"]["nome"], x["fields"]["regiao"]) for x in municipios], columns=["IBGE", "municipio", "regiao"])

TypeError: string indices must be integers

In [ ]:
municipios.to_pickle("dados/municipios.pd.pkl")

In [ ]:
municipios.to_dict(orient="records")

[{'ibge': 4200051, 'municipio': 'Abdon Batista', 'regiao': 'Meio Oeste'},
 {'ibge': 4200101, 'municipio': 'Abelardo Luz', 'regiao': 'Xanxerê'},
 {'ibge': 4200200, 'municipio': 'Agrolândia', 'regiao': 'Alto Vale do Itajaí'},
 {'ibge': 4200309, 'municipio': 'Agronômica', 'regiao': 'Alto Vale do Itajaí'},
 {'ibge': 4200408, 'municipio': 'Água Doce', 'regiao': 'Meio Oeste'},
 {'ibge': 4200507, 'municipio': 'Águas de Chapecó', 'regiao': 'Oeste'},
 {'ibge': 4200556, 'municipio': 'Águas Frias', 'regiao': 'Oeste'},
 {'ibge': 4200606,
  'municipio': 'Águas Mornas',
  'regiao': 'Grande Florianópolis'},
 {'ibge': 4200705,
  'municipio': 'Alfredo Wagner',
  'regiao': 'Grande Florianópolis'},
 {'ibge': 4200754,
  'municipio': 'Alto Bela Vista',
  'regiao': 'Alto Uruguai Catarinense'},
 {'ibge': 4200804, 'municipio': 'Anchieta', 'regiao': 'Extremo Oeste'},
 {'ibge': 4200903, 'municipio': 'Angelina', 'regiao': 'Grande Florianópolis'},
 {'ibge': 4201000,
  'municipio': 'Anita Garibaldi',
  'regiao': '

In [ ]:
{ slugfy(x): x for x in municipios.regiao.unique()}

{'meio-oeste': 'Meio Oeste',
 'xanxere': 'Xanxerê',
 'alto-vale-do-itajai': 'Alto Vale do Itajaí',
 'oeste': 'Oeste',
 'grande-florianopolis': 'Grande Florianópolis',
 'alto-uruguai-catarinense': 'Alto Uruguai Catarinense',
 'extremo-oeste': 'Extremo Oeste',
 'serra-catarinense': 'Serra Catarinense',
 'medio-vale-do-itajai': 'Médio Vale do Itajaí',
 'nordeste': 'Nordeste',
 'extremo-sul-catarinense': 'Extremo Sul Catarinense',
 'laguna': 'Laguna',
 'alto-vale-do-rio-do-peixe': 'Alto Vale do Rio do Peixe',
 'foz-do-rio-itajai': 'Foz do Rio Itajaí',
 'planalto-norte': 'Planalto Norte',
 'carbonifera': 'Carbonífera'}

In [ ]:
municipios.query("ibge == 4202901")

,ibge,municipio,regiao


In [14]:
populi = pd.read_csv("dados/populacao.csv")

In [38]:
populi.query("IBGE in [4202404,4219705]").sum().to_dict()

{'IBGE': 8422109,
 'Município': 'Blumenau (SC)Xaxim (SC)',
 '2013': 356121,
 '2014': 361338,
 '2015': 366506,
 '2016': 371636,
 '2017': 376723,
 '2018': 380884,
 '2019': 385905,
 '2020': 390838,
 '2021': 395672}

In [41]:
#populi[populi.Município.str.match("Blumenau.*")]
mun = pd.read_pickle("dados/municipios.pd.pkl")

In [44]:
lista_ibge = mun.query("regiao == 'Meio Oeste'").ibge.unique()

'IBGE in [4200051, 4200408, 4202875, 4203600, 4203907, 4204004, 4204152, 4205209, 4206702, 4206801, 4208609, 4209003, 4209201, 4210035, 4211058, 4211801, 4218509, 4219150, 4219176, 4219853]'

In [60]:
populi.query(f"IBGE in {list(lista_ibge)}").sum().to_dict()

{'IBGE': 84174104,
 'Município': "Abdon Batista (SC)Água Doce (SC)Brunópolis (SC)Campos Novos (SC)Capinzal (SC)Catanduvas (SC)Celso Ramos (SC)Erval Velho (SC)Herval d'Oeste (SC)Ibicaré (SC)Jaborá (SC)Joaçaba (SC)Lacerdópolis (SC)Luzerna (SC)Monte Carlo (SC)Ouro (SC)Treze Tílias (SC)Vargem (SC)Vargem Bonita (SC)Zortéa (SC)",
 '2013': 186274,
 '2014': 187422,
 '2015': 188555,
 '2016': 189682,
 '2017': 190801,
 '2018': 191303,
 '2019': 192347,
 '2020': 193375,
 '2021': 194381}

In [59]:
populi.query("IBGE == 4202404").to_dict(orient="records")[0]

{'IBGE': 4202404,
 'Município': 'Blumenau (SC)',
 '2013': 329082,
 '2014': 334002,
 '2015': 338876,
 '2016': 343715,
 '2017': 348513,
 '2018': 352460,
 '2019': 357199,
 '2020': 361855,
 '2021': 366418}

In [56]:
populi.loc[36]

IBGE               4202404
Município    Blumenau (SC)
2013                329082
2014                334002
2015                338876
2016                343715
2017                348513
2018                352460
2019                357199
2020                361855
2021                366418
Name: 36, dtype: object

Ibge
4200051      1
4200101     60
4200200    142
4200309      3
4200408    118
          ... 
4219507    533
4219606      6
4219705     97
4219853      1
4220000    368
Name: Asma, Length: 288, dtype: int64

In [80]:
series = df.query("ano == 2016").groupby("Ibge").sum()["Asma"]/populi.set_index("IBGE")["2016"]*1000
series.to_dict()

{4200051: 0.38211692777990064,
 4200101: 3.374198627825891,
 4200200: 13.618490457466194,
 4200309: 0.5585552038726495,
 4200408: 16.51966960660787,
 4200507: 11.852776044915784,
 4200556: 0.8343763037129746,
 4200606: 29.97099581050596,
 4200705: 18.576476527006562,
 4200754: 3.540718259989884,
 4200804: 11.507869351836183,
 4200903: 15.006002400960384,
 4201000: 4.151530877010898,
 4201109: 11.996308828052907,
 4201208: 8.999148729174268,
 4201257: 24.25230061349693,
 4201273: 16.596540439457687,
 4201307: 6.614108128857,
 4201406: 11.905120255260227,
 4201505: 4.388566006404933,
 4201604: 0.8417508417508417,
 4201653: 20.741394527802296,
 4201703: 15.199897815813003,
 4201802: 4.589963280293758,
 4201901: nan,
 4201950: 4.840594224670339,
 4202008: 1.2222247527082528,
 4202057: 3.7724610344485257,
 4202073: 10.564770932069509,
 4202081: 0.3598416696653472,
 4202099: 15.306122448979592,
 4202107: 11.669128508124077,
 4202131: 2.5493945188017846,
 4202156: 0.0,
 4202206: 12.5697087722

In [82]:
contornos = load_json("dados/contornos-municipios.json")

In [89]:
pre_ibge = df.Ibge.apply(lambda x: x//10)

In [90]:
df.insert(1, "ibg", pre_ibge)

In [103]:
pre_chaveamento = df[["Ibge", "ibg"]].to_dict(orient="records")

In [104]:
chaveamento = {x['ibg']: x["Ibge"] for x in pre_chaveamento}

In [109]:
for x in contornos["features"]:
    ibg = x["properties"]["id"]
    ibge = chaveamento[ibg]
    x["properties"]["id"] = ibge


In [112]:
from main import dump_json

In [ ]:
dump_json("dados/contornos-municipios.json")